## **Contenido**

[**1. Perceptron multi-capa**](#tema1)

[**2. Perceptron multi-capa en pytorch**](#tema2)

[**3. Redes Neuronales Convolucionales**](#tema3)


[**4. Preguntas**](#tema7)

## <font color='#4C5FDA'>**Segmentación de imagenes vasculares de la retina** </font> <a name="tema1">

In [ ]:
! pip install kaggle segmentation-models-pytorch

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torchvision.datasets import ImageFolder

from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import datasets, models, transforms, utils
import torchvision.transforms.v2 as transforms

import segmentation_models_pytorch as smp
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import albumentations as A
from PIL import Image
from glob import glob
from tqdm import tqdm

In [ ]:
! kaggle datasets download -d abdallahwagih/retina-blood-vessel
! unzip retina-blood-vessel.zip

In [ ]:
train_img_paths = sorted(glob('/content/Data/train/image/*.png'))
train_mask_paths = sorted(glob('/content/Data/train/mask/*.png'))
train_df = pd.DataFrame({"images":train_img_paths,"masks":train_mask_paths})
train_df.head()

In [ ]:
test_img_paths = sorted(glob('/content/Data/test/image/*.png'))
test_mask_paths = sorted(glob('/content/Data/test/mask/*.png'))
test_df = pd.DataFrame({"images":test_img_paths,"masks":test_mask_paths})
test_df.head()

In [ ]:
show_imgs = 8
idx = np.random.choice(len(train_df), show_imgs, replace=False)
fig, axes = plt.subplots(show_imgs*2//4, 4, figsize=(10, 8))
axes = axes.flatten()
for i, ax in enumerate(axes):
    new_i = i//2
    if i % 2 ==0 :
        full_path = train_df.loc[idx[new_i]]['images']
        basename = os.path.basename(full_path)
    else:
        full_path = train_df.loc[idx[new_i]]['masks']
        basename = os.path.basename(full_path) + ' -mask'
    ax.imshow(plt.imread(full_path))
    ax.set_title(basename)
    ax.set_axis_off()

In [ ]:
### Elige trasnformaciones para hacer data augmentation
### Puedes encontrar informacion acerca de las transformaciones disponibles en https://albumentations.ai/docs/api_reference/augmentations/
train_transforms = A.Compose([

])

test_transforms = A.Compose([

])

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, transforms_=None):
        self.df = dataframe
        self.transforms_ = transforms_
        self.resize = [256, 256]
        self.class_size = 2

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img = cv2.cvtColor(cv2.imread(self.df.iloc[index]['images']), cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.df.iloc[index]['masks'],cv2.IMREAD_GRAYSCALE)
        mask = np.where(mask<127, 0, 1).astype(np.int16)
        aug = self.transforms_(image=img, mask=mask)
        img, mask = aug['image'], aug['mask']
        img = img/255

        img = torch.tensor(img, dtype=torch.float).permute(2, 0, 1)
        target = torch.tensor(mask, dtype=torch.long)
        sample = {'image': img, 'mask': target}
        return sample

In [ ]:
train_dataset = #Invoca la clase MyDataset declarada para el dataset de entrenamiento
val_dataset = #Invoca la clase MyDataset declarada para el dataset de validacion

BATCH_SIZE = None
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)
print(f'len train: {len(train_df)}')
print(f'len val: {len(test_df)}')

len train: 80
len val: 20


In [ ]:
class DoubleConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels):
     super().__init__()
     self.double_conv = nn.Sequential(
         #Declara las capas necesarias para realizar la doble convolución descrita en el diagrama de la UNet
     )

  def forward(self, x):
    return self.double_conv(x)


class Downscaling(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()
    self.maxpool_conv = ##Añade el max pooling al bloque de doble convolución

  def forward(self, x):
    return self.maxpool_conv(x)


class Upscaling(nn.Module):
  def __init__(self, in_channels, out_channels, bilinear=True):
    super().__init__()

    ## Declara las capas necesarias para aumentar la escala de los mapas de caracteristicas
    ## Puedes emplear interpolación o convolución transpuesta para lograrlo

  def forward(self, x1, x2):
      x1 = self.up(x1)
      # input is CHW
      diffY = x2.size()[2] - x1.size()[2]
      diffX = x2.size()[3] - x1.size()[3]

      x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                      diffY // 2, diffY - diffY // 2])

      x = torch.cat([x2, x1], dim=1)
      return self.conv(x)


class OutConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()
    self.conv = #declara la capa de salida segun las clases de tu dataset

  def forward(self, x):
    return self.conv(x)

In [ ]:
class UNet(nn.Module):
  def __init__(self, n_channels, n_classes, bilinear=False):
    super().__init__()

    self.n_channels = None
    self.n_classes = None
    self.bilinear = None

    ## Siguiendo la estructura de la Unet, crea el modelo con los bloques declarados arriba

  def forward(self, x):

    ## Crea el forward de la UNet, no olvides las conexiones residuales

    return logits

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNet(n_channels=3, n_classes=1)

model.to(device)
criterion = smp.losses.DiceLoss(mode="binary")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=5)
epochs = None

for epoch in range(epochs+1):
  model.train()
  epoch_loss = 0
  epoch_iou_score = 0
  train_iters = 0

  for _, batch in enumerate(tqdm(train_loader, desc=f'Epoch {epoch}/{epochs}')):

    images, true_mask = batch['image'].to(device), batch['mask'].to(device)
    optimizer.zero_grad()

    pred_masks = model(images)
    pred = pred_masks.squeeze(dim=1)

    loss = criterion(pred, true_mask)
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
    true_mask = true_mask.round().long()
    tp, fp, fn, tn = smp.metrics.get_stats(pred, true_mask, mode='binary', threshold=0.5)
    iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro").item()
    epoch_iou_score += iou_score
    train_iters += 1

  print(f"Epoch Loss: {epoch_loss/train_iters:.4f}, Dice: {loss:.4f}, IoU: {epoch_iou_score:.4f}")

In [ ]:
model.eval()
epoch_loss = None
train_iters = None
epoch_iou_score = None

with torch.no_grad():
    for _, batch in enumerate(tqdm(val_loader, desc=f'Epoch {epoch}/{epochs}')):
        images, true_masks = batch['image'].to(device), batch['mask'].to(device)

        pred = model(images)
        loss = criterion(pred, true_masks)

        epoch_loss += loss.item()
        pred = pred.squeeze(dim=1)
        pred = torch.sigmoid(pred)
        true_mask = true_mask.round().long()
        tp, fp, fn, tn = smp.metrics.get_stats(pred, true_mask, mode='binary', threshold=0.5)
        iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro").item()
        epoch_iou_score += iou_score
        train_iters += 1

print(f"Epoch Loss: {epoch_loss/train_iters:.4f}, Dice: {loss:.4f}, IoU: {epoch_iou_score/train_iters:.4f}")

In [ ]:
#images, masks = next(iter(val_loader))
images, masks = batch['image'].to(device), batch['mask'].to(device)
with torch.no_grad():
  model_prediction = model(images)

model_prediction = torch.max(model_prediction,dim=1)[0]

rows, columns = 2, 6
num_images = 24
fig, axs = plt.subplots(rows, columns, figsize=(15, 6))
idx = 0
for row in range(rows):
    for col in range(0, 6, 3):
      if idx >= num_images:
          break

      img = images[idx].permute(1, 2, 0)
      mask = masks[idx].squeeze()
      predicted_masks = model_prediction[idx]


      axs[row, col].imshow(img.cpu())
      axs[row, col].set_title('Input image')
      axs[row, col].axis('off')

      axs[row, col+1].imshow(mask.cpu())
      axs[row, col+1].set_title('Target Mask')
      axs[row, col+1].axis('off')

      axs[row, col+2].imshow(predicted_masks.cpu())
      axs[row, col+2].set_title('Predicted Mask')
      axs[row, col+2].axis('off')

      idx += 1

plt.tight_layout()
plt.show()